# UnpackAI Library Development Plan
> What library should we be?
This proposal, is more of a branding proposal, targeting people who's going to play with AI, from various back grounds.
* That means, we're going to talk about how people view this library, how they think of ```pip install -Uqq unpackai``` like if I have dandroff recengtly and my mind just jump right into the headshoulders.
* For ML, currently, the **jump** is about the following, this is not a throught marketing research, just quick examples from a deep learning practitioner:
    * Try free structure quickly, do experiments: pytorch
    * Goes to production, run model on edge devices, Tensorflow
    * Play with GPU accelerated tensor calculation: Jax
    * Play with tf but in simpler layer sense: Keras
    * Transformer in clean code: Huggingface
    * Visualize things with interactive features: Plotly
    * Deploy model prototype: streamlit
* Surely you think I fail to mention ```fastai```, this is where the **branding goes wrong**, fastai library is bounded tightly with the education. It's considered a good creation along side its famous course, after the education. Its product feature has many limitation: docs too brief, not supporting multi-device training, very limited numbers of callbacks went beyond Jeremy H's own teaching.
* Most important of all, ```fastai``` isn't enjoyable to use, **it's just packing many things mentioned in the course**.

## What we shouldn't be
I know the course is life changing for me and I feel very grateful. But let's not be their library.

### The pipeline wrapping plan
It all started from a notebook, quite like a template notebook we have for the course. A notebook that achieves the data processing, model building, interpretation for a specific DL task.

Then came the packaging part, we wrap **dozens of lines of codes**, which scares our kind students, into simple functions, or class.

The wrapped functions are simple to use, to look at, it was executed in 1 line mostly. So friendly to our innocent students.

This is what a python library is about, right? Wrap things into functions which can be further wraped into even less lines.

It's nothing wrong about this approach at first. Some DL task, if need be, can be shrank into **less than 10 lines of codes.**
* The 1st line load the data, 
* the 2nd line set how to transform data, 
* the 3rd line build/load the model, 
* the 4th line trained model.
* the 5th line interpret the model in various ways

Well the above do look like a decent **structure** to start with, then we pave out the tasks, different contributors take different tasks, can be developed in parallel, and we can have the agile/crum/kanban fun to track our progress!

Even if we do this, we could build a useful product, no less.

#### Bad side about pipeline wrapping plan
So so many libraries are doing the same, from awesome people even. They usually end up to the following:
* It's a mess of functions, among them many good functions but a mess. It ends up a branding disaster. (**There is no way to answer: what can you library do, in a slogan**)
* A model zoo for a specific domain.
* Wraping things up means less and less involvement from the user. The user will spend very little time play with the functions, and each function usually achieve very specific task. Actually I do believe there is a equilibrium like:
$\large{UserPlayHours = a * Task Transferability}$

## Alternative approach

The salvation plan is somehow simpler at how we perceive the library:
* A library that allows you experiment AI/DL for various tasks

**BUT!!!**
* Many module with in the pipeline should be dropdown-list/checkbox **Choosable**.
* The **level of detail** we let them to play and choose, is the **level of the difficulty** we want them to enjoy

### What is level of detail ?
Level of details is the level of fuss we want user to focus on, this is the exact part fastai library got **WRONG**, which will explain most of our struggle so far:
* It offers smooth/ easy pipelines, for newbies and business people even.
* Any amount of reconfigure, is usually way too complicated for such audience
* There is a **GAP** between the 2 points above, hence no room for playing

#### Keras Example 
I started my AI journey with Keras, and I love keras by that time, because:
* Keras plays with **layers**(eg. Linear, Convolution), its most strenth is at astracting details beneath this level, and let users play with layers. 
* I spent lots of time, having fun playing with layers
* Aside from the things I have to redesign layer, I can deploy almost all kinds of models mentioned in any DL paper (𝑈𝑠𝑒𝑟𝑃𝑙𝑎𝑦𝐻𝑜𝑢𝑟𝑠=𝑎∗𝑇𝑎𝑠𝑘𝑇𝑟𝑎𝑛𝑠𝑓𝑒𝑟𝑎𝑏𝑖𝑙𝑖𝑡𝑦)

#### Pytorch lightning example
Well I moved on to the career team. I have to deal with layer level, I have to deal with different data/forward pipeline. PL is a good library because:
* It allows me play with the things I mentioned, but save my energy on things like looping, logging, multidevice training detail etc.
* If you see a training notebook built by PL, you'll see very little lines around training template.
* You'll find about a lots of lines on the specifications you intend to be different.

>The branding image of the examples are simple:
* Keras: play TensorFlow in a concept of layers
* Pytorch-Lightning: writting less template code

#### Unpackai Example
For our lib, I intend for them to focus on, exactly the same range of things we want people to learn:
* choose the columns they intend to use, in what way
* choose the data transformations
* choose the loss, the model structure to use (not keras.layer, not nn.module)
* hit run

## Demo of such example

In [1]:
from ipywidgets import interact, interact_manual
from forgebox.imports import *
from forgebox.category import Category

In [2]:
import pytorch_lightning as pl

In [3]:
HOME = Path(os.environ['HOME'])

Let's skip data download here, it's download, we're not going to reinvent brilliant stuff around download

In [4]:
# BEAR_DATASET = HOME/"Downloads"/"bear_dataset"
BEAR_DATASET = "/GCI/data/bear_dataset"

### Step 1 Everything starts with dataframe

For fastai, everything starts from list, an ItemList to be specific. **ImageList** and **TextList** is [**ItemList**](https://fastai1.fast.ai/tutorial.itemlist.html) with some slight enhanced feature.```[🧂, 🏓, 🍷, 🐻]```

For the clarity of education, or for simplecity as ultimate form of beauty, we use [**DataFrame**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) as starting point, ItemList in table format. In this way, every dataset has the same starting point, even the tabular data. 

In [5]:
def df_creator_image_folder(path: Path):
    path = Path(path)
    files = list(path.rglob("*.jpg"))
    files.extend(path.rglob("*.JPG"))
    files.extend(path.rglob("*.jpeg"))
    files.extend(path.rglob("*.JPEG"))
    files.extend(path.rglob("*.png"))
    files.extend(path.rglob("*.PNG"))
    return pd.DataFrame({"path":files}).sample(frac=1.).reset_index(drop=True)

## Enrich columns (feature transformation, label extraction)
After this step, there will only be **MORE** column ➕

### Phase/ Configuration

In [6]:
from typing import List, Dict, Callable, Any, Tuple
from torchvision import transforms as tfm
from PIL import Image
from forgebox.html import DOM
from ipywidgets import VBox, HBox, HTML

In [7]:
from ipywidgets import (
    Text, Textarea, IntSlider, FloatSlider, SelectMultiple, Dropdown, Checkbox,
    Layout, Button
)
from typing import List, Dict, Any

In [8]:
class Phase:
    """
    A configuration management mechanism
    """
    is_phase = True
    def __init__(self, **kwargs):
        self.config = dict()
        self.config.update(kwargs)
        
    def __setitem__(self, k, v):
        self.config[k] = v
    
    def __getitem__(self, k):
        return self.config[k]
    
    def __contains__(self, k):
        return k in self.config
    
    def __call__(self):
        return self.get_data(self.config)
    
    def get_data(self, raw):
        """
        Reconstruct back to dict or list or value format
        """
        if hasattr(raw,"is_phase"):
            return raw.get_data(raw.config)
        if type(raw) == list:
            raw = list(self.get_data(i) for i in raw)
            return raw
        if type(raw) == dict:
            for k, v in raw.items():
                raw[k] = self.get_data(v)
            return raw
        return raw

    def __str__(self):
        return json.dumps(self(), indent=2)
    
    def __repr__(self,):
        return f"Phase:{self}"
    

class EnrichPhase(Phase):
    def __init__(self, *steps):
        super().__init__()
        self.config['steps'] = []
        for step in steps:
            checked = self.check_step(step)
            if checked:
                self.config['steps'].append(checked)
                
    def new_step(self, process, dst:str, src: str=None):
        self.config['steps'].append({
            "process":process,
            "src":src,
            "dst":dst
        })
    
    def check_step(self,step):
        return step

### Widget Helpers

#### More or less

In [9]:
class MoreOrLess(VBox):
    """
    Interactive list
    You can add item to the list
    Each added item has a remove button to remove such item
    """
    def __init__(self, data_list: List[Any] = []):
        super().__init__([])
        for data in data_list:
            self+data

    def create_line(self, data):
        children = list(self.children)
        children.append(self.new_line(data))
        self.children = children

    @staticmethod
    def data_to_dom(data):
        return HTML(json.dumps(data))

    def new_line(self, data) -> HBox:
        del_btn = Button(description="Remove", icon="trash")
        del_btn.button_style = 'danger'
        hbox = HBox([del_btn, self.data_to_dom(data)])
        hbox.data = data

        def remove_hbox():
            children = list(self.children)
            for i, c in enumerate(children):
                if id(c) == id(hbox):
                    children.remove(c)
            self.children = children
        del_btn.click = remove_hbox
        return hbox

    def __add__(self, data):
        self.create_line(data)
        return self

    def get_data(self):
        """
        Return the data of this list
        """
        return list(x.data for x in self.children)

In [10]:
mol = MoreOrLess()

In [11]:
mol+{"hello":"yes"}

MoreOrLess(children=(HBox(children=(Button(button_style='danger', description='Remove', icon='trash', style=Bu…

### typings for interactives

In [12]:
class InteractiveTyping:
    """
    Typing for interactive details
    self.__call__() will create widgets directly
    """
    name = "anything"
    is_typing = True

    def solid(self, default) -> None:
        """
        Reset default value
        """
        if default is not None:
            self.default = default


class INT(InteractiveTyping):
    def __init__(self, min_: int = 0, max_: int = 10, step: int = 1, default: int = None):
        self.max_ = max_
        self.min_ = min_
        self.step = step
        self.default = default if default is not None else 1

    def __repr__(self):
        return f"int[{self.min_}-{self.max_}, :{self.step}]={self.default}"

    def __call__(self, default: int = None):
        self.solid(default)
        return IntSlider(
            value=self.default,
            min=self.min_,
            max=self.max_,
            step=self.step,
        )


class BOOL(InteractiveTyping):
    def __init__(self, name:str="", default: bool = True,):
        self.default = default
        self.name = name

    def __repr__(self):
        return f"bool={self.default}"

    def __call__(self, default: bool = None) -> Checkbox:
        self.solid(default)
        return Checkbox(value=self.default, description=self.name)


class FLOAT(InteractiveTyping):
    def __init__(self, min_: int = -1., max_: int = 1., step: int = .01, default: int = None):
        self.max_ = max_
        self.min_ = min_
        self.step = step
        self.default = default if default is not None else 0.01

    def __repr__(self):
        return f"float[{self.min_}-{self.max_}, :{self.step}]={self.default}"

    def __call__(self, default: int = None):
        self.solid(default)
        return FloatSlider(
            value=self.default,
            min=self.min_,
            max=self.max_,
            step=self.step,
        )


class STR(InteractiveTyping):
    """
    String object
    will create text or textarea
    """

    def __init__(self, default: str = None, use_area: bool = False):
        """
        use_area: do we use Textarea, if False,we use Text
        """
        self.default = "" if default is None else default
        self.use_area = use_area

    def __repr__(self):
        return f"str='{self.default}'"

    def __call__(self, default: str = None):
        self.solid(default)
        if self.use_area:
            return Textarea(value=self.default, layout=Layout(width="80%"))
        return Text(value=self.default)


class LIST(InteractiveTyping):
    """
    dropdown list type or multiselection type
    """

    def __init__(self, options: List[Any] = [], default: Any = None, multi: bool = False):
        """
        if multi: default should be iterable
        else: default should be one of the option
        """
        self.options = options
        self.default = default
        self.multi = multi

    def __repr__(self):
        if self.multi:
            size = f"[0-{self.default}]/{len(self.options)}"
        else:
            size = f"1/{len(self.options)}"
        return f"list,{size}"

    def __call__(self, default: Any = None):
        self.solid(default)
        if self.multi:
            inter = SelectMultiple(options=self.options)
        else:
            inter = Dropdown(options=self.options)

        if self.default is not None:
            # if multi: default should be iterable
            # else: default should be one of the option
            inter.value = self.default
        return inter

class InteractiveAnnotations:
    """
    Build interactive based on the info of function's ```__annotations__```
    """

    def __init__(
        self, func: Callable,
        icon: str = "rocket",
        description: str = 'Run',
        button_style='primary'
    ):
        self.func = func
        self.icon = icon
        self.button_style = button_style
        self.description = description
        self.build_vbox(func)

    @classmethod
    def on(
        cls,
        callback: Callable,
        icon: str = 'rocket',
        description: str = 'Run',
        button_style: str = 'primary'
    ) -> Callable:
        """
        Use this class as a decorator
        @InteractiveAnnotation.on(callback)
        def target_func(a:STR(), b:INT()=1):
            ...
        """
        def decorator(func: Callable):
            obj = cls(
                func,
                icon=icon,
                description=description,
                button_style=button_style
            )
            display(obj.vbox)
            obj.register_callback(callback=callback)
            return func
        return decorator

    def build_vbox(self, func: Callable):
        row_list = []
        self.fields = dict()
        for k, v in func.__annotations__.items():
            if hasattr(v, "is_typing") == False:
                continue
            widget = v()
            widget.description = k
            row_list.append(widget)
            self.fields.update({k: widget})

        # final button
        self.final_btn = Button(
            description=self.description,
            icon=self.icon,
        )
        self.final_btn.button_style = self.button_style
        row_list.append(self.final_btn)

        # create interactive
        self.vbox = VBox(row_list)
        return self.vbox

    def register_callback(
        self,
        callback: Callable
    ) -> None:
        def run_callback():
            kwargs = self()
            callback(kwargs)
        self.final_btn.click = run_callback

    def __call__(self) -> Dict[str, Any]:
        """
        extract interactive data values
        """
        rt = dict()
        for k, widget in self.fields.items():
            rt.update({k: widget.get_interact_value()})
        return rt

In [13]:
def print_stuff(kwargs):
    print(kwargs)

@InteractiveAnnotations.on(print_stuff, "flask", "test", button_style="warning")
def some_func(e, a:STR(), b:INT()=2, d=3):
    print(1)

In [14]:
STR('RGB')()

Text(value='RGB')

### Enrich

In [15]:
class Enrich:
    """
    Enrich Base Class
    Some default attributes
    - is_enrich = True
    - typing = None # output typing
    - multi_cols = False # use multi-column as input
    - prefer = None
    - lazy = False  # shall we execute enrichment only through the iteration
    - src = None # source column
    """
    is_enrich = True
    typing = None # output typing
    multi_cols = False # use multi-column as input
    prefer = None
    lazy = False  # shall we execute enrichment only through the iteration
    src = None # source column

    def __init__(self): pass

    def __call__(self, row):
        return row
    
    def rowing(self, row):
        if self.multi_cols:
            return self(row)
        else:
            return self(row[self.src])


class EnrichImage(Enrich):
    """
    Create Image column from image path column
    """
    prefer = "QuantifyImage"
    typing = Image
    lazy = True
    

    def __init__(
        self, convert: STR("RGB") = "RGB",
        size: LIST(options=[28, 128, 224, 256, 512], default=224) = 224,
    ):
        self.convert = convert
        self.size = size

    def __repr__(self):
        return f"[Image:{self.size}]"

    def __call__(self, x):
        img = Image.open(x).convert(self.convert)
        img = img.resize((self.size, self.size))
        return img


class ParentAsLabel(Enrich):
    typing = str
    prefer = "QuantifyCategory"
    def __call__(self, path: Path,) -> str:
        """
        Use parent folder name as label
        """
        return Path(path).parent.name
    
ENRICHMENTS = dict(
    EnrichImage=EnrichImage,
    ParentAsLabel=ParentAsLabel,
)

### Interact creation

In [16]:
def print_kwargs(kwargs):
    print(kwargs)
    return kwargs


def reconfig_manual_interact(
    widget,
    description: str = "Create",
    button_style: str = "primary",
    icon: str = "plus"
) -> Button:
    """
    reconfigure the button of interactive features
    """
    btn = None
    for w in widget.children:
        if type(w) == Button:
            btn = w
            break
    btn.description = description
    btn.button_style = button_style
    btn.icon = icon
    return btn


def interact_intercept(
    func:Callable,
    result_cb: Callable = print_kwargs
):
    """
    Initialize a class with interactive features
    """
    annotations = func.__annotations__
    defaults = func.__defaults__
    kwargs = dict()
    if defaults is not None:
        for (k, typing), default in zip(annotations.items(), defaults):
            kwargs.update({k: typing(default)})
    obj = dict()

    def fillin_init(**kwargs):
        obj.update({
            "kwargs": kwargs,
        })
    f = interact_manual(fillin_init, **kwargs)

    btn = reconfig_manual_interact(f.widget)

    if btn is not None:
        original = btn.click

        def new_click_event():
            original()
            return result_cb(obj['kwargs'])
        btn.click = new_click_event

    return obj, f

def init_interact(cls, result_cb: Callable = print_kwargs):
    return interact_intercept(cls.__init__, result_cb=result_cb)

In [17]:
obj,f = init_interact(EnrichImage)

interactive(children=(Text(value='RGB', description='convert'), Dropdown(description='size', index=2, options=…

In [18]:
phase = Phase()

In [19]:
bear_df = df_creator_image_folder(BEAR_DATASET)

bear_df

,path
0,/GCI/data/bear_dataset/teddys/00000026.jpg
1,/GCI/data/bear_dataset/black/00000135.jpg
2,/GCI/data/bear_dataset/teddys/00000062.jpeg
3,/GCI/data/bear_dataset/black/00000062.jpg
4,/GCI/data/bear_dataset/black/00000044.jpg
...,...
517,/GCI/data/bear_dataset/grizzly/00000011.jpg
518,/GCI/data/bear_dataset/black/00000099.jpg
519,/GCI/data/bear_dataset/grizzly/00000167.jpg
520,/GCI/data/bear_dataset/grizzly/00000047.jpg


In [22]:
def set_enrich(df):
    global phase
    DOM(f"{len(df)} rows of data, example table", "h3")()
    display(df.sample(5))
    display(HTML("<hr>"))

    def setting_col():
        enrich_data_list = phase['enrich'] if 'enrich' in phase else []
        enrich_box = MoreOrLess(enrich_data_list)
        display(enrich_box)

        
        def set_enrich_(src=["[all_columns]", ]+list(df.columns)):
            DOM(f"Setting up column enrich: {src}", "h4")()
            if src == "[all_columns]":
                display(df.head(3))
            else:
                display(df[[src, ]].head(3))

            def choose_enrich(dst="", enrich=ENRICHMENTS):
                DOM(f"Source: {src}, Destination: {dst}, for {enrich.__name__}", "h4")(
                )
                DOM(f"{enrich.__doc__}", "quote")()

                def result_callback(kwargs):
                    extra = {"src": src, "dst": dst,
                                "kwargs": kwargs, "enrich": enrich.__name__}
                    enrich_box+extra
                    phase['enrich'] = enrich_box.get_data()
                obj, decoed_func = init_interact(enrich, result_callback)
            choose_enrich_widget = interact_manual(choose_enrich).widget
            reconfig_manual_interact(
                choose_enrich_widget,
                description="Choose", button_style='warning')
        set_enrich_widget = interact_manual(set_enrich_).widget
        reconfig_manual_interact(set_enrich_widget, button_style='warning')
    setting_col()

In [24]:
set_enrich(bear_df)

,path
23,/GCI/data/bear_dataset/black/00000144.jpg
399,/GCI/data/bear_dataset/teddys/00000096.jpg
299,/GCI/data/bear_dataset/black/00000109.jpg
28,/GCI/data/bear_dataset/grizzly/00000094.jpg
335,/GCI/data/bear_dataset/black/00000069.jpg


HTML(value='<hr>')

MoreOrLess()

interactive(children=(Dropdown(description='src', options=('[all_columns]', 'path'), value='[all_columns]'), B…

In [25]:
phase

Phase:{
  "enrich": [
    {
      "src": "path",
      "dst": "image",
      "kwargs": {
        "convert": "RGB",
        "size": 224
      },
      "enrich": "EnrichImage"
    },
    {
      "src": "path",
      "dst": "label",
      "kwargs": {},
      "enrich": "ParentAsLabel"
    }
  ]
}

In [26]:
from tqdm.notebook import tqdm

### Execute enrichment

In [27]:
def execute_enrich(
    df: pd.DataFrame, phase:Phase
):
    for en_conf in tqdm(phase["enrich"], leave=False):
        enrich_name = en_conf['enrich']
        enrich_cls = ENRICHMENTS[enrich_name]
        kwargs = en_conf['kwargs']
        src = en_conf['src']
        dst = en_conf['dst']
        # The class with lazy loading, will only 
        # call the class only if necessary
        if enrich_cls.lazy:
            obj = enrich_cls(**kwargs)
            obj.src = src
            df[dst] = obj
        # The class without lazy loading
        # create the column now
        else:
            obj = enrich_cls(**kwargs)
            if src=="[all_columns]":
                df[dst] = df.apply(obj, axis=1)
            else:
                df[dst] = df[src].apply(obj)
    return df

In [28]:
execute_enrich(bear_df, phase)

  0%|          | 0/2 [00:00<?, ?it/s]

,path,image,label
0,/GCI/data/bear_dataset/teddys/00000026.jpg,[Image:224],teddys
1,/GCI/data/bear_dataset/black/00000135.jpg,[Image:224],black
2,/GCI/data/bear_dataset/teddys/00000062.jpeg,[Image:224],teddys
3,/GCI/data/bear_dataset/black/00000062.jpg,[Image:224],black
4,/GCI/data/bear_dataset/black/00000044.jpg,[Image:224],black
...,...,...,...
517,/GCI/data/bear_dataset/grizzly/00000011.jpg,[Image:224],grizzly
518,/GCI/data/bear_dataset/black/00000099.jpg,[Image:224],black
519,/GCI/data/bear_dataset/grizzly/00000167.jpg,[Image:224],grizzly
520,/GCI/data/bear_dataset/grizzly/00000047.jpg,[Image:224],grizzly


### Quantify: Choose columns as X and Y, put them into number

In [29]:
from transformers import AutoTokenizer

In [30]:
class SIZE_DIMENSION:
    pass

class BATCH_SIZE(SIZE_DIMENSION):
    def __repr__(self): return f"BATCH_SIZE"

class SEQUENCE_SIZE(SIZE_DIMENSION):
    pass

class IMAGE_SIZE(SIZE_DIMENSION):
    pass

In [31]:
class Quantify:
    is_quantify = True
    """
    # From all things to number
    The AI model does not understand anything, say, picture, text
    Unless you transform it to integer and float tensors

    Quantify and its subclass controls the
        numericalization / collation of the data pipeline
    The base class of quantify does: NOTHING
    """

    def __init__(self,):
        pass

    def __call__(self, list_of_items):
        return list(list_of_items)

    def adapt(self, column):
        """
        A function to let the data processing
        adapt to the data column
        """
        pass
    
    def __hash__(self,):
        if hasattr(self, "name"):
            return self.name
        else:
            return self.__class__.__name__


class QuantifyImage(Quantify):
    """
    Transform PIL.Image to tensor
    """

    def __init__(
        self,
        mean_: LIST(["imagenet", "0.5 x 3"]) = "imagenet",
        std_: LIST(["imagenet", "0.5 x 3"]) = "imagenet",
    ):
        if type(mean_) == str:
            if mean_ == "imagenet":
                mean_ = [0.485, 0.456, 0.406]
            elif mean_ == "0.5 x 3":
                mean_ = [.5, .5, .5]
            else:
                raise ValueError(
                    f"Mean configuration: {mean_} not valid")

        if type(std_) == str:
            if std_ == "imagenet":
                std_ = [0.229, 0.224, 0.225]
            elif std_ == "0.5 x 3":
                std_ = [.5, .5, .5]
            else:
                raise ValueError(
                    f"Standard Variation configuration: {std_} not valid")

        self.transform = tfm.Compose([
            tfm.ToTensor(),
            tfm.Normalize(mean=mean_, std=std_),
        ])
        
        self.shape = (BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE)

    def __repr__(self):
        return f"Quantify Image to tensors:{self.transform}"

    def __call__(self, list_of_image):
        return torch.stack(list(
            self.transform(img) for img in list_of_image))


class QuantifyText(Quantify):
    def __init__(
        self,
        pretrained: STR(default="bert-base-cased") = "bert-base-cased",
        max_length: INT(default=512, min_=12, max_=1024, step=4) = 512,
        padding: LIST(options=[
            "do_not_pad",
            "max_length",
            "longest"], default="max_length") = "max_length",
        return_token_type_ids: BOOL(name="Token Type IDs", default=True) = True,
        return_attention_mask: BOOL(name="Attention Mask", default=True) = True,
        return_offsets_mapping: BOOL(name="Offset Mapping", default=False) = False,
    ):
        self.pretrained = pretrained
        self.max_length = max_length
        self.padding = padding
        self.return_token_type_ids = return_token_type_ids
        self.return_attention_mask = return_attention_mask
        self.return_offsets_mapping = return_offsets_mapping
        self.truncation = True
        self.return_tensors = 'pt'
        self.shape = (BATCH, SEQUENCE_SIZE)

    def adapt(self, column):
        """
        Initialize tokenizer
        """
        from transformers import AutoTokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.pretrained, use_fast=True)

    def __call__(self, list_of_text: List[str]):
        list_of_text = list(list_of_text)
        return self.tokenizer(
            list_of_text,
            padding=self.padding,
            max_length=self.max_length,
            truncation=self.truncation,
            return_token_type_ids=self.return_token_type_ids,
            return_attention_mask=self.return_attention_mask,
            return_tensors=self.return_tensors,
            return_offsets_mapping=self.return_offsets_mapping,
        )


class QuantifyCategory(Quantify):
    """
    Transform single categorical data to index numbers in pytorch tensors
    """

    def __init__(
        self,
        min_frequency: INT(min_=1, max_=20, default=1) = 1,
    ):
        self.min_frequency = min_frequency

    def adapt(self, column):
        # category statistics
        value_counts = pd.DataFrame(column.value_counts())

        # if minimun freq is 1
        # very category occured should be accounted for
        # hence no missing token padding is required
        if self.min_frequency < 2:
            self.category = Category(
                arr=np.array(value_counts.index),
                pad_mst=False)

        # we need missing token
        # for category's frequency < self.min_frequency
        else:
            categories = np.array(
                list(value_counts.index[
                    value_counts.values.reshape(-1) > self.min_frequency]))
            self.category = Category(arr=categories, pad_mst=True)

        self.shape = (BATCH_SIZE, len(self.category))

    def __repr__(self):
        return f"Quantify Category:{self.category}"

    def __call__(self, list_of_strings):
        return torch.LongTensor(self.category.c2i[np.array(list_of_strings)])


class QuantifyNum(Quantify):
    """
    Quantify contineous data, like float numbers
    The only process is normalization on the entire population
    """
    shape = (BATCH_SIZE, 1)
    def adapt(self, column):
        self.mean_ = column.mean()
        self.std_ = column.std()

    def __call__(self, list_of_num):
        return (torch.FloatTensor(list_of_num)[None,:]-self.mean_)/self.std_

    def backward(self, x):
        return x*self.std_+self.mean_


class QuantifyMultiCategory(Quantify):
    """
    Turn Multi-categorical data to n_hot encoding numbers in pytorch tensors
    """

    def __init__(self, col_name: str):
        self.col_name = col_name


QUANTIFY = dict(
    Quantify=Quantify,
    QuantifyImage=QuantifyImage,
    QuantifyCategory=QuantifyCategory,
    QuantifyMultiCategory=QuantifyMultiCategory,
    QuantifyText=QuantifyText,
)

### Pytorch Dataset

In [32]:
class TaiChiDataset(Dataset):
    """
    A pytorch dataset working under our environment
    """
    def __init__(self, df, columns: List[Any] = None):
        self.df = df
        self.columns = list(df.columns) if columns is None else columns

    def __len__(self):
        return len(self.df)

    def shuffle(self):
        self.df = self.df.sample(frac=1.).reset_index(drop=True)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        row = dict(self.df.loc[idx])
        rt = dict()
        for col in self.columns:
            v = row[col]
            if hasattr(v, "is_enrich"):
                rt[col] = v.rowing(row)
            else:
                rt[col] = v
        return rt
    
    def split(
        self,
        valid_ratio:FLOAT(min_=0.01, max_=0.5, default=.1, step=0.01)=.1
    ) -> Tuple[Any]:
        cls = self.__class__
        slicing = (np.random.rand(len(self)) < valid_ratio)
        return (
            cls(self.df[slicing].reset_index(drop=True), self.columns),
            cls(self.df[~slicing].reset_index(drop=True), self.columns)
        )

    def dataloader(
        self,
        batch_size: LIST(options=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512], default=32) = 32,
        shuffle: LIST(options=[True, False], default=False) = False,
        num_workers: LIST(options=[0, 2, 4, 8, 16], default=0) =0,
    ):
        return DataLoader(
            self,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers)

In [33]:
from forgebox.html import list_group_kv

In [34]:
ds = TaiChiDataset(bear_df)
ds[1]

{'path': PosixPath('/GCI/data/bear_dataset/black/00000135.jpg'),
 'image': <PIL.Image.Image image mode=RGB size=224x224 at 0x7FC1D8579C50>,
 'label': 'black'}

### Choose XY

In [35]:
def choose_xy(df):
    global phase
    DOM(f"{len(df)} rows of data, example table", "h3")()
    display(df.sample(5))
    display(HTML("<hr>"))
    DOM("Please Choose Column", "h3")()
    DOM("The AI model will try to guess the Y with the input X", "div", {"style":"color:#666699"})()
    
    task = 'quantify'
    # enrich by columns
    if "enrich" in phase:
        by_destination = dict((en['dst'], en) for en in phase['enrich'])
    else:
        by_destination = dict()
    
    data_list = phase[task] if task in phase else []
    mol_box = MoreOrLess(data_list)
    display(mol_box)

    @interact_manual
    def set_quantify_(src=list(df.columns), use_for = ["As X", "As Y"]):
        DOM(f"Quantify Column: {src} {use_for}", "h4")()
        display(df[[src, ]].head(3))
        
        quantify_dropdown = Dropdown(options=list(QUANTIFY.keys()))
        
        # check the hint from last step
        prefer = None
        if src in by_destination:
            col_config = by_destination[src]
            cls = ENRICHMENTS[col_config['enrich']]

            # In case the enrich layer has the preference
            if hasattr(cls, "prefer"):
                prefer = cls.prefer
                
                # set default value to drop down value,
                # if the the previous hint suggest so
                quantify_dropdown.value = prefer
                DOM(f"Prefered quantifying:\t{cls.prefer}", "h4")()
            if hasattr(cls, "typing"):
                DOM(f"Output data type:\t{cls.typing}", "h4")()
        
        @interact_manual
        def choose_quantify(quantify = quantify_dropdown):
            cls = QUANTIFY[quantify]
            def result_callback(kwargs):
                extra = {"src": src, "x":(use_for=="As X"),
                        "kwargs": kwargs, "quantify": cls.__name__}
                mol_box+extra
                phase['quantify'] = mol_box.get_data()
                
            obj, decoded = init_interact(cls, result_callback)

In [36]:
choose_xy(bear_df)

,path,image,label
55,/GCI/data/bear_dataset/teddys/00000182.jpg,[Image:224],teddys
268,/GCI/data/bear_dataset/black/00000106.jpg,[Image:224],black
269,/GCI/data/bear_dataset/grizzly/00000049.jpg,[Image:224],grizzly
54,/GCI/data/bear_dataset/teddys/00000092.jpg,[Image:224],teddys
116,/GCI/data/bear_dataset/black/00000177.jpg,[Image:224],black


HTML(value='<hr>')

MoreOrLess()

interactive(children=(Dropdown(description='src', options=('path', 'image', 'label'), value='path'), Dropdown(…

In [37]:
phase

Phase:{
  "enrich": [
    {
      "src": "path",
      "dst": "image",
      "kwargs": {
        "convert": "RGB",
        "size": 224
      },
      "enrich": "EnrichImage"
    },
    {
      "src": "path",
      "dst": "label",
      "kwargs": {},
      "enrich": "ParentAsLabel"
    }
  ],
  "quantify": [
    {
      "src": "image",
      "x": true,
      "kwargs": {
        "mean_": "imagenet",
        "std_": "imagenet"
      },
      "quantify": "QuantifyImage"
    },
    {
      "src": "label",
      "x": false,
      "kwargs": {
        "min_frequency": 1
      },
      "quantify": "QuantifyCategory"
    }
  ]
}

In [66]:
def execute_quantify(
    df: pd.DataFrame, phase:Phase
):
    # existance check
    if 'quantify' not in phase:
        raise KeyError(f"No quantify stepset")
    
    qdict = dict()
    for i, qconf in tqdm(enumerate(phase['quantify']), leave = False):
        qname = qconf['quantify']
        kwargs = qconf['kwargs']
        src = qconf['src']
        x = qconf['x']
        
        cls = QUANTIFY[qname]
        qobj = cls(**kwargs)
        qobj.src = src
        qobj.is_x = x
        qobj.adapt(df[src])
        qdict.update({src:qobj})
    return qdict

In [67]:
qdict = execute_quantify(bear_df, phase)

0it [00:00, ?it/s]

In [68]:
qdict

{'image': Quantify Image to tensors:Compose(
     ToTensor()
     Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
 ),
 'label': Quantify Category:Category Manager with 3}

### Create collate with dict

In [74]:
class TaiChiCollate:
    """
    Universal all power full collate function
    """
    def __init__(self, quantify_dict):
        self.quantify_dict = quantify_dict
        
    def make_df(self, batch):
        return pd.DataFrame(list(batch))
        
    def __len__(self):
        return len(self.quantify_dict)
        
    def __call__(self, batch):
        batch_df = self.make_df(batch)
        rt = dict()
        for src,qobj in self.quantify_dict.items():
            rt.update({src:qobj(list(batch_df[src]))})
        return rt

In [75]:
import pytorch_lightning as pl

In [76]:
class TaiChiDataModule(pl.LightningDataModule):
    def __init__(self, dataset: TaiChiDataset, quantify_dict: Dict[str, Quantify]):
        super().__init__()
        self.dataset = dataset
        self.quantify_dict = quantify_dict
        
        self.collate = TaiChiCollate(quantify_dict)
        
    def configure(
        self,
        valid_ratio:FLOAT(min_=0.01, max_=0.5, default=.1, step=0.01)=.1,
        batch_size: LIST(options=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512], default=32) = 32,
        shuffle: LIST(options=[True, False], default=False) = True,
        num_workers: LIST(options=[0, 2, 4, 8, 16], default=0) =0,
    ):  
        self.train_ds, self.val_ds = self.dataset.split(valid_ratio)
        self.batch_size=batch_size
        self.shuffle=shuffle
        self.num_workers=num_workers
        
    def train_dataloader(self):
        self.train_dl = self.train_ds.dataloader(
            batch_size=self.batch_size,
            shuffle=self.shuffle,
            num_workers=self.num_workers)
        self.train_dl.collate_fn = self.collate
        return self.train_dl
    
    def val_dataloader(self):
        self.val_dl = self.val_ds.dataloader(
            batch_size=self.batch_size,
            shuffle=self.shuffle,
            num_workers=self.num_workers)
        self.val_dl.collate_fn = self.collate
        return self.val_dl

In [77]:
def execute_datamodule(df, qdict, phase):
    ds = TaiChiDataset(df)
    datamodule = TaiChiDataModule(ds, qdict)
    
    def configure_setting(kwargs):
        global phase
        datamodule.configure(**kwargs)
        phase['batch_level'] = kwargs
    interact_intercept(datamodule.configure, configure_setting)
    return datamodule

In [78]:
datamodule = execute_datamodule(bear_df, qdict, phase)

interactive(children=(FloatSlider(value=0.1, description='valid_ratio', max=0.5, min=0.01, step=0.01), Dropdow…

In [79]:
data = next(iter(datamodule.train_dataloader()))

###  Choose your model, loss

In [81]:
phase

Phase:{
  "enrich": [
    {
      "src": "path",
      "dst": "image",
      "kwargs": {
        "convert": "RGB",
        "size": 224
      },
      "enrich": "EnrichImage"
    },
    {
      "src": "path",
      "dst": "label",
      "kwargs": {},
      "enrich": "ParentAsLabel"
    }
  ],
  "quantify": [
    {
      "src": "image",
      "x": true,
      "kwargs": {
        "mean_": "imagenet",
        "std_": "imagenet"
      },
      "quantify": "QuantifyImage"
    },
    {
      "src": "label",
      "x": false,
      "kwargs": {
        "min_frequency": 1
      },
      "quantify": "QuantifyCategory"
    }
  ],
  "batch_level": {
    "valid_ratio": 0.1,
    "batch_size": 32,
    "shuffle": true,
    "num_workers": 0
  },
  "x_models": {
    "image": {
      "model_name": "ImageConvEncoder",
      "src": "image",
      "kwargs": {
        "name": "resnet18"
      }
    }
  },
  "y_models": {
    "label": {
      "model_name": "CategoryTop",
      "src": "label",
      "kwargs": {

In [82]:
from torchvision.models import (
    resnet18, resnet34, resnet50, resnet101, resnet152,
    resnext101_32x8d, resnext50_32x4d)

In [53]:
# class MultiCategoricalEmbedding(nn.Module):
#     def __init__(
#         self,num_embeddings,
#         embedding_dim: LIST([2,4,8,16,32,64,128,256,512,1024], default=64,)
#     ):
#         super().__init__()
#         self.emb = nn.Embedding(num_embeddings, embedding_dim)
        
#     def forward(self, x):
        
        

## Models

### Entry modules

In [99]:
RESNET_OPTIONS = {"resnet18": resnet18,
                  "resnet34": resnet34,
                  "resnet50": resnet50,
                  "resnet101": resnet101,
                  "resnet152": resnet152,
                  "resnext101_32x8d": resnext101_32x8d,
                  "resnext50_32x4d": resnext50_32x4d}

In [55]:
class MidJoint1d(nn.Module):
    def __init__(self, keys):
        super().__init__()
        self.keys = keys
    
    def forward(self, data):
        tensors = list(data[key] for key in self.keys)
        return torch.cat(tensors,dim=1)

In [100]:
class EntryModel(nn.Module):
    is_entry = True
    
class Empty(EntryModel):
    def __init__(self):
        super().__init__()
        self.out_features=1
    
    def forward(self, x):
        return x
    
    @classmethod
    def from_quantify(cls,
        quantify):
        return cls()

class ImageConvEncoder(EntryModel):
    def __init__(self, model):
        super().__init__()
        self.name = "cnn"
        self.output_shape = (BATCH_SIZE, model.fc.in_features)
        self.out_features = model.fc.in_features
        model.fc = Empty()
        self.model = model

    def forward(self, data):
        return self.model(data)

    def __repr__(self):
        return f"""ComputerVisionEncoder: {self.name}
        Outputs shape:{self.output_shape}"""

    @classmethod
    def from_quantify(
        cls,
        quantify,
        name: LIST(options=list(
            RESNET_OPTIONS.keys()), default="resnet18"),
    ):
        model = RESNET_OPTIONS[name](pretrained=True, progress=True,)
        obj = cls(model)
        obj.name = name
        return obj


class CategoryEncoder(EntryModel):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        self.model = nn.Embedding(
            num_embeddings,
            embedding_dim)
        
    def forward(self, idx):
        return self.model(idx)
    
    @classmethod
    def from_quantify(
        quantify,
        embedding_dim: LIST(
            options=[4, 8, 16, 32, 64, 128, 256, 512], default=128) = 128):
        num_embeddings = len(quantify.category)
        obj = CategoryEncoder(
            num_embeddings=num_embeddings,
            embedding_dim=embedding_dim,
        )
        obj.out_features = out_features
        return obj


class TransformerEncoder(EntryModel):
    """
    A model part to encode sequnce data in to vectors
    """

    def __init__(self, model, encoder_mode: BOOL(default=True) = True,):
        super().__init__()
        self.model = model
        self.encoder_mode = encoder_mode

    def forward(self, kwargs):
        outputs = self.model(**kwargs)
        if self.encoder_mode:
            # output vector
            return outputs.logits*kwargs['attention_mask'].mean(1)
        return outputs

    @classmethod
    def from_quantify(
        cls,
        quantify,
        name: STR(default="bert-base-uncased") = 'bert-base-cncased',
        encoder_mode: BOOL(default=True) = True,
    ):
        from transformer import AutoModel
        model = AutoModel.from_pretrained(name)
        obj = cls(model)
        obj.name = name
        obj.encoder_mode = encoder_mode
        if encoder_mode:
            obj.out_features= model.config.hidden_size
        return obj

In [57]:
entry = ImageConvEncoder.from_quantify(0,name="resnet18")

In [58]:
entry

ComputerVisionEncoder: resnet18
        Outputs shape:(<class '__main__.BATCH_SIZE'>, 512)

In [59]:
with torch.no_grad():
    vectors = entry(data['image'])

### Exit modules

In [101]:
class ExitModel(nn.Module):
    def loss_step(self, x, y):
        y_ = self(x)
        loss = self.crit(y_, y)
        return {"loss": loss, "y_": y_}
    
class CategoryTop(ExitModel):
    prefer = "CrossEntropyLoss"
    input_dim = 2

    def __init__(self, in_features, out_features):
        super().__init__()
        self.top = nn.Linear(
            in_features=in_features, out_features=out_features)
        self.activation = nn.Softmax(dim=-1)
        self.crit = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.top(x)
    
    @classmethod
    def from_quantify(cls, quantify, entry_part):
        out_features = len(quantify.category)
        in_features = entry_part.out_features
        return cls(
            in_features=in_features,
            out_features=out_features,
        )

class MultiCategoryTop(ExitModel):
    prefer = "BCEWithLogitsLoss"
    input_dim = 2

    def __init__(self, in_features, out_features):
        super().__init__()
        self.top = nn.Linear(
            in_features=in_features, out_features=out_features)
        self.activation = nn.Sigmoid()
        self.crit = nn.BCEWithLogitsLoss()

    def forward(self, x):
        return self.top(x)
    
    def loss_step(self, x, y):
        y_ = self(x)
        loss = self.crit(y_, y)
        return {"loss": loss, "y_": self.activation(y_)}
    
    @classmethod
    def from_quantify(cls, quantify, entry_part):
        out_features = len(quantify.category)
        in_features = entry_part.out_features
        return cls(
            in_features=in_features,
            out_features=out_features,
        )


class RegressionTop(ExitModel):
    prefer = "MSELoss"
    input_dim = 2

    def __init__(self, in_features, out_features):
        super().__init__()
        self.top = nn.Linear(
            in_features=in_features, out_features=out_features)
        self.crit = nn.MSELoss()

    def forward(self, x):
        return self.top(x)
    
    @classmethod
    def from_quantify(cls, quantify, entry_part):
        out_features = 1
        in_features = entry_part.out_features
        return cls(
            in_features=in_features,
            out_features=out_features,
        )

### EntireModel

In [102]:
QUANTIFY_2_ENTRY_MAP = dict({
    QuantifyImage:[
        ImageConvEncoder,
    ],
    QuantifyCategory:[
        CategoryEncoder,
    ],
    QuantifyText:[
        TransformerEncoder,
    ],
    QuantifyNum:[
        Empty,
    ],
})
ENTRY_ALL = dict(
    ImageConvEncoder=ImageConvEncoder,
    CategoryEncoder=CategoryEncoder,
    TransformerEncoder=TransformerEncoder,
    Empty=Empty,
)

QUANTIFY_2_EXIT_MAP = dict({
    QuantifyCategory:[
        CategoryTop,
    ],
    QuantifyNum:[
        RegressionTop,
    ],
})
EXIT_ALL = dict(
    CategoryTop=CategoryTop,
    RegressionTop=RegressionTop,
)

In [62]:
def choose_models(quantify, cls_options, model_conf:str='x_models'):
    def config_model(ModelClass=cls_options):
        def starting_cls(kwargs):
            global phase
            if model_conf in phase:
                models = phase[model_conf]
            else:
                models = dict()
            models[quantify.src] = dict(
            model_name=ModelClass.__name__,
            src=quantify.src,
            kwargs=kwargs,
            )
            phase[model_conf] = models
        ia = InteractiveAnnotations(
            ModelClass.from_quantify,
            description="Okay",
            icon='rocket',
            button_style='success')
            
        ia.register_callback(starting_cls)
        display(ia.vbox)
    inter = interact_manual(config_model)
    reconfig_manual_interact(
        inter.widget,
        description="Yes!", icon="cube", button_style='info')
    return inter
    

In [86]:
def set_model(quantify_dict: Dict[str, Quantify]):
    global phase
    x_models = dict()
    y_models = dict()
    for src, quantify in quantify_dict.items():
        if quantify.is_x:
            entry_cls_options = dict(
                (q.__name__, q)
                for q in QUANTIFY_2_ENTRY_MAP.get(quantify.__class__))

            if entry_cls_options is None:
                print(f"We do not support {quantify.__class__} as X data")
                continue
            display(HTML(f"""
            <h3 class='text-primary'>Choose Model For X Column:
            <strong>{src}</strong></h3>"""))
            choose_models(quantify, entry_cls_options, "x_models")
    for src, quantify in quantify_dict.items():
        if quantify.is_x == False:
            exit_cls_options = dict(
                (q.__name__, q)
                for q in QUANTIFY_2_EXIT_MAP.get(quantify.__class__))
            if entry_cls_options is None:
                print(f"We do not support {quantify.__class__} as Y data")
            display(HTML(f"""
            <h3 class='text-danger'>Choose Model For Y Column:
            <strong>{src}</strong></h3>"""))
            choose_models(quantify, exit_cls_options, "y_models")

In [87]:
set_model(qdict)

HTML(value="\n            <h3 class='text-primary'>Choose Model For X Column:\n            <strong>image</stro…

interactive(children=(Dropdown(description='ModelClass', options={'ImageConvEncoder': <class '__main__.ImageCo…

HTML(value="\n            <h3 class='text-danger'>Choose Model For Y Column:\n            <strong>label</stron…

interactive(children=(Dropdown(description='ModelClass', options={'CategoryTop': <class '__main__.CategoryTop'…

In [88]:
phase

Phase:{
  "enrich": [
    {
      "src": "path",
      "dst": "image",
      "kwargs": {
        "convert": "RGB",
        "size": 224
      },
      "enrich": "EnrichImage"
    },
    {
      "src": "path",
      "dst": "label",
      "kwargs": {},
      "enrich": "ParentAsLabel"
    }
  ],
  "quantify": [
    {
      "src": "image",
      "x": true,
      "kwargs": {
        "mean_": "imagenet",
        "std_": "imagenet"
      },
      "quantify": "QuantifyImage"
    },
    {
      "src": "label",
      "x": false,
      "kwargs": {
        "min_frequency": 1
      },
      "quantify": "QuantifyCategory"
    }
  ],
  "batch_level": {
    "valid_ratio": 0.1,
    "batch_size": 32,
    "shuffle": true,
    "num_workers": 0
  },
  "x_models": {
    "image": {
      "model_name": "ImageConvEncoder",
      "src": "image",
      "kwargs": {
        "name": "resnet18"
      }
    }
  },
  "y_models": {
    "label": {
      "model_name": "CategoryTop",
      "src": "label",
      "kwargs": {

In [113]:
class EntryDict(nn.Module):
    def __init__(
        self,
        phase: Phase,
        qdict: Dict[str, EntryModel]
    ):
        super().__init__()
        model_dict = nn.ModuleDict()
        for src, model_cfg in phase['x_models'].items():
            quantify = qdict[src]
            model_cls = ENTRY_ALL[model_cfg['model_name']]
            model_kwargs = model_cfg['kwargs']
            model = model_cls.from_quantify(quantify, **model_kwargs)
            model_dict[src] = model

        self.out_features = sum(
            list(model.out_features for src, model in model_dict.items()))
        self.model_dict = model_dict

    def forward(self, inputs):
        outputs = []
        for src, model in self.model_dict.items():
            src_input = inputs[src]
            outputs.append(model(src_input))
        return torch.cat(outputs, dim=1)

In [114]:
entry_dict = EntryDict(phase, qdict)

In [115]:
with torch.no_grad():
    y_ = entry_dict(data)

In [116]:
y_.shape

torch.Size([32, 512])

In [149]:
class AssembledModel(pl.LightningModule):
    def __init__(
        self,
        phase: Phase,
        qdict: Dict[str, EntryModel],
        entry_lr: LIST(options=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6], default=1e-4)=1e-4,
        exit_lr: LIST(options=[1e-1, 1e-2, 1e-3, 1e-4, ], default=1e-3)=1e-3,
    ):
        super().__init__()
        self.entry_lr = entry_lr
        self.exit_lr = exit_lr
        self.entry_dict = EntryDict(phase, qdict)
        exit_cfg = list(phase['y_models'].values())[0]
        
        self.exit_src = exit_cfg['src']
        self.exit_kwargs = exit_cfg['kwargs']
        exit_cls = EXIT_ALL[exit_cfg['model_name']]
        
        exit_quantify = qdict[self.exit_src]
        
        self.exit_part = exit_cls.from_quantify(
            exit_quantify,self.entry_dict, **self.exit_kwargs)
    
    def forward(self, inputs):
        vec = self.entry_dict(inputs)
        return self.exit_part(vec)
    
    def loss_step(self, inputs):
        vec = self.entry_dict(inputs)
        return self.exit_part.loss_step(vec, inputs[self.exit_src])
    
    def training_step(self, batch, batch_idx):
        rt = self.loss_step(batch)
        for k, v in rt.items():
            self.log(f"trn_{k}", v)
        return rt['loss']

    def validation_step(self, batch, batch_idx):
        rt = self.loss_step(batch)
        for k, v in rt.items():
            self.log(f"val_{k}", v)
        return rt['loss']
    
    def configure_optimizers(self,):
        param_groups = [
            {"params":self.entry_dict.parameters(), "lr":self.entry_lr},
            {"params":self.exit_part.parameters(), "lr":self.exit_lr},
        ]
        return torch.optim(param_groups)
    

In [144]:
def create_model(phase, qdict):
    if "y_models" in phase:
        y_models = phase["y_models"]
        if len(y_models)>1:
            raise ValueError("Multiple targets are not supported by now")
        else:
            return AssembledModel(phase, qdict)
    else:
        raise ValueError("phase must contain 'y_models' configuration for now")

In [145]:
final_model = create_model(phase, qdict)

In [147]:
with torch.no_grad():
    final_model.loss_step(data)

### Training

In [ ]:
class Module